In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Model
class recommendeur:
    '''
    Recommandation distances cosinus    
    '''
    
    
    '''
        Constructeur avec jeu de donnees : chargement du jeux de donnees
        
        params :
        meta : pandas dataframe avec colonnes: [id, noms]
        ratings : pandas dataframe avec colonnes : [user id, film id, rating]
        
        * note : les user id range de 1,...,number_user
    '''
    def __init__(self, meta, ratings, rating_missing_value=2.5):
        self.meta = meta
        self.ratings = ratings
        
        # calcul du nombre de films
        self.number_films = len(ratings.iloc[:,1].unique())
        
        # calcul du nombre d'utilisateur
        self.number_user = len(ratings.iloc[:,0].unique())
        
        # dictionnaire de conversion index film : film_id -> index [0 , ... , number_films - 1]
        self.index = {meta.iloc[i,0]:i for i in range(self.number_films)}
        self.index_nom = {meta.iloc[i,1]:i for i in range(self.number_films)}
        
        # Matrice des features : films sont les observations et les ratings des user sont les features
        self.X = rating_missing_value*np.ones((self.number_films, self.number_user))
        for i in range(len(ratings)):
            if ratings.iloc[i,2] != -1 and ratings.iloc[i,1] in self.index:
                self.X[ self.index[ratings.iloc[i,1]], ratings.iloc[i,0]-1 ] = ratings.iloc[i,2]
                
    def calcul_distance(self, film1, film2):
        return np.sum(self.X[film1]*self.X[film2])/(np.linalg.norm(self.X[film1])*np.linalg.norm(self.X[film2]))
    
    '''  Recommander des nouveau films
    
         deja_vue : liste de titre de film deja appreciee
         nombre : nombre de recommendations
         option : et/ou
             et : retourn des films qui sont en liens avec tout les films deja_vu
             ou : retourn des films qui sont en liens avec au moins un des films deja vu
    '''
    def recommander(self, deja_vue, nombre=4, option='ou'): 
        idx = [self.index_nom[film] for film in deja_vue]
        
        d = -1000+np.zeros(self.number_films)
        for j in range(self.number_films):
            if j not in idx:
                if option=='ou':
                    similarite = 0
                elif option=='et':
                    similarite = 1
                
                for i in idx:
                    if option=='ou':
                        similarite += self.calcul_distance(i,j)
                    elif option=='et':
                        similarite *= self.calcul_distance(i,j)
                d[j] = similarite
                
        rec_idx = np.argsort(d)[-nombre:]
        return [self.meta.iloc[ri,1] for ri in rec_idx]

In [3]:
# Charger les donnes des titres
animes_meta = pd.read_csv('anime.csv', usecols=[0,1])
movies_meta = pd.read_csv('movies_metadata.csv', usecols=[5, 8])
movies_id = pd.read_csv('movies_links_small.csv')

In [4]:
# Charger les ratings
animes_ratings = pd.read_csv('animes_rating.csv')
movies_ratings = pd.read_csv('movies_ratings_small.csv', usecols=[0,1,2])

In [5]:
# Reduction de la taille du jeu de donnees animes : 1000 users
animes_ratings_reduit = animes_ratings[animes_ratings['user_id'] <= 1000]

In [6]:
# conversion des index imdb pour movies_meta
def isfloat(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

for i in range(len(movies_meta)):
    #print(i)
    if isfloat(movies_meta.iloc[i,0]) and  (movies_id['tmdbId']==float(movies_meta.iloc[i,0])).sum()==1: 
        movies_meta.at[i,'id'] = int(movies_id['movieId'][movies_id['tmdbId']==float(movies_meta.iloc[i,0]) ].iloc[0] )

In [7]:
# Definition de l'objet du recommendeur pour animes
r1 = recommendeur(animes_meta, animes_ratings_reduit)

In [11]:
# Entrer des films déjà vus et appréciés (une liste)
# Voir les noms exacte dans movies_metadata.csv
r1.recommander(['One Piece'], 3, option='et')

['Doraemon: The Day When I Was Born',
 'Battle Spirits: Brave',
 'Kagaku Ninja-tai Gatchaman (Movie)']

In [9]:
# Definition de l'objet du recommendeur pour film
r2 = recommendeur(movies_meta, movies_ratings, rating_missing_value=1.0)

In [12]:
# Entrer des films déjà vus et appréciés (une liste)
# Voir les noms exacte dans movies_metadata.csv
r2.recommander(['Ice Age', 'Finding Nemo'], 3, option='et')

['Johnny English', "Something's Gotta Give", 'Shark Tale']